In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'
#this script classifies station precip by weather type

In [ ]:
#Taiwan bounds: 0-50 N,80-160 E
#NEUSA bounds: 10-60 N,30-115 W
#These are large enough that an AR invading the NEUSA or Taiwan would be detected - i.e. >2000 km on all sides
#ds=xr.open_dataset(dir+'ls_extreme_rain_taiwan.nc')
#dates=ds.large_scale_extreme_days

ds2=xr.open_dataset(dir+'ivt_basics_neusa_95_spring.nc')#change this threshold for sensitivity testing
#ds2['longitude']=longitude_sub

#Winter  425.82147
#Spring 448.2696
#Summer 534.098
#Fall 528.6943
#Warm 531.6989

ivt_threshold=531.6989

ivt_mag=ds2['ivt_mag']
ivtx=ds2['ivtx']
ivty=ds2['ivty']
lons_full=ivt_mag.longitude.values
lats_full=ivt_mag.latitude.values
print(ivt_mag)

lat_range=np.arange(0,60,0.25)
lon_range=np.arange(-140,-110,0.25)

In [ ]:
ds1=xr.open_dataset(dir+'station_numbers_95.nc')
lats_station=ds1['lats'].values.tolist()#use these lists for plotting
lons_station=ds1['lons']
lons_station=lons_station.values.tolist()
stations1=ds1['stations'].values.tolist()
thresholds=ds1['thresholds_annual_99']/10.
print(stations1)

In [ ]:
if 1==1:#part 2: read the obs at eligible stations into a better format
    yr_start=1979
    yr_end=2019
    tuples=[]
    years=[]
    months=[]
    days1=[]
    lats1=[]
    lons1=[]
    elevs1=[]
    observations=[]
    dates=[]
    station_names_list=[]
    stations_list=[]
    dp=xr.open_dataset(dir+'station_numbers_95.nc')
    stations=dp.stations.values.tolist()
    lats=dp.lats.values.tolist()
    lons=dp.lons.values.tolist()
    elevs=dp.elevs.values.tolist()
    station_names=dp.station_names.values.tolist()
    points=zip(lons,lats)
    for i in range(len(stations)):
        print(str(i+1)+'/'+str(len(stations)))
        lat=lats[i]
        lon=lons[i]
        elev=elevs[i]
        station_name=station_names[i]
        station=stations[i]
        filename=stations[i]
        ds1=pd.read_csv(filename,header=None)
        nancount=0
        qcount=0
        yescount=0
        for row1 in ds1.iterrows():
            row_neg_space=row1[1][0].replace('-',' -')
            row_basic=row1[1][0]
            a=row_neg_space.split()
            code=a[0][-4:]
            if code=='PRCP':
                year=int(a[0][11:15])
                if yr_start<=year<=yr_end:
                    month=int(a[0][15:17])
                    if month in [1,3,5,7,8,10,12]:
                        days=31
                    if month in [4,6,9,11]:
                        days=30
                    if month==2 and year%4==0:
                        days=29
                    if month==2 and year%4!=0:
                        days=28

                    for i in range(days):
                        #print(row_basic)
                        mflag=row_basic[26+8*i]
                        qflag=row_basic[27+8*i]
                        sflag=row_basic[28+8*i]
                        obs=float(row_basic[21+8*i:26+8*i])
                    
                        if obs==-9999.0:
                            pass
                        elif qflag!=' ':
                            pass
                        else:
                            years.append(year)
                            months.append(month)
                            days1.append(i)
                            observations.append(obs/10.)#convert from tenths of mm to mm
                            lats1.append(lat)
                            lons1.append(lon)
                            elevs1.append(elev)
                            date=dt.datetime(year,month,i+1,6)
                            dates.append(date)
                            station_names_list.append(station_name)
                            stations_list.append(station)
#lons1=[x+360. for x in lons1]

In [ ]:
ds6=xr.open_dataset(dir+'IBTrACS.NA.v04r00.nc')
#print(ds)
lat_tc=ds6.lat
lon_tc=ds6.lon
#nature_tc=ds6.nature
ds7=xr.open_dataset(dir+'ibtracs_na_time.nc')
years_tc=ds7.years
months_tc=ds7.months
days_tc=ds7.days
hours_tc=ds7.hours
nature_tc=ds7.nature#1 if tropical, 0 if not
print(nature_tc)
print(hours_tc)

In [ ]:
dsa=xr.open_dataset(dir+'neusa_ar_points_springwarm_var95const.nc')
dsa['longitude']=dsa.longitude
ar=dsa.ar
tc_trop=dsa.tc_trop
tc_nontrop=dsa.tc_nontrop
labeled=dsa.labeled

In [ ]:
print(ivt_threshold)
import matplotlib as mpl

import warnings
warnings.filterwarnings('ignore')#ignore warnings

lats_era5=ivt_mag.latitude.values.tolist()
lons_era5=ivt_mag.longitude.values.tolist()

zipped=list(zip(dates,lats1,lons1,observations,stations_list))

yrs_neusa=np.arange(1979,2020,1)
season='spring'
for l in range(27,len(yrs_neusa)):
    year=yrs_neusa[l]
    if season=='winter':
        if year%4==0:
            feb_length=29
        else:
            feb_length=28
        base_date_1=dt.datetime(year-1,12,1,6)
        dates_list_1=[base_date_1+dt.timedelta(days=x) for x in range(31)]

        base_date_2=dt.datetime(year,1,1,6)
        dates_list_2=[base_date_2+dt.timedelta(days=x) for x in range(31+feb_length)]

        dates_list=dates_list_1+dates_list_2

    if season=='spring':
        base_date=dt.datetime(year,3,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(31+30+31)]

    if season=='summer':
        base_date=dt.datetime(year,6,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(30+31+31)]

    if season=='fall':
        base_date=dt.datetime(year,9,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(30+31+30)]

    dates_unique=dates_list
    print(dates_unique)

    lats_tc_ar_combo=[]
    lons_tc_ar_combo=[]
    stations_tc_ar_combo=[]
    obs_tc_ar_combo=[]
    dates_tc_ar_combo=[]

    lats_pure_tc=[]
    lons_pure_tc=[]
    stations_pure_tc=[]
    obs_pure_tc=[]
    dates_pure_tc=[]

    stations_tc_remnant_ar_combo=[]
    lats_tc_remnant_ar_combo=[]
    lons_tc_remnant_ar_combo=[]
    obs_tc_remnant_ar_combo=[]
    dates_tc_remnant_ar_combo=[]

    stations_tc_linked_ar=[]
    lats_tc_linked_ar=[]
    lons_tc_linked_ar=[]
    obs_tc_linked_ar=[]
    dates_tc_linked_ar=[]

    stations_tc_remnant_linked_ar=[]
    lats_tc_remnant_linked_ar=[]
    lons_tc_remnant_linked_ar=[]
    obs_tc_remnant_linked_ar=[]
    dates_tc_remnant_linked_ar=[]

    stations_pure_ar=[]
    lats_pure_ar=[]
    lons_pure_ar=[]
    obs_pure_ar=[]
    dates_pure_ar=[]

    stations_tc_remnants=[]
    lats_tc_remnants=[]
    lons_tc_remnants=[]
    obs_tc_remnants=[]
    dates_tc_remnants=[]

    stations_tc_remnant_linked_ivt=[]
    lats_tc_remnant_linked_ivt=[]
    lons_tc_remnant_linked_ivt=[]
    obs_tc_remnant_linked_ivt=[]
    dates_tc_remnant_linked_ivt=[]

    stations_tc_linked_ivt=[]
    lats_tc_linked_ivt=[]
    lons_tc_linked_ivt=[]
    obs_tc_linked_ivt=[]
    dates_tc_linked_ivt=[]

    stations_pure_extreme_ivt=[]
    lats_pure_extreme_ivt=[]
    lons_pure_extreme_ivt=[]
    obs_pure_extreme_ivt=[]
    dates_pure_extreme_ivt=[]

    stations_other=[]
    lats_other=[]
    lons_other=[]
    obs_other=[]
    dates_other=[]

    for d in range(len(dates_unique)):
        date=dates_unique[d]#date is at 6Z automatically.  IVT goes from 6Z on day to 6Z on next day
        print(date)
        date_save=date
        select_date=[x for x in zipped if x[0]==date]
        date_midday=date+dt.timedelta(hours=12)

        date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
        date_range=pd.DatetimeIndex(date_range).values#date range for IVT, AR, EP comparison

        #here: daily field calculations
        ivt_mag_date_four=ivt_mag.sel(time=date_range)
        ivt_mag_date=ivt_mag_date_four.max(dim='time')
        extreme_ivt_date=ivt_mag_date.where(ivt_mag_date>=ivt_threshold)
        ar_date_four=ar.sel(time=date_range)
        ar_max_four=ar_date_four.max(dim='time')
        labeled_four=labeled.sel(time=date_range)

        #here: calculations at each station
        for i in range(len(stations1)):
            lat=lats_station[i]
            lon=lons_station[i]
            station=stations1[i]
            obs_station=[x[3] for x in select_date if x[4]==station]

            if len(obs_station)>0:
                obs_station=obs_station[0]
                int_lat=int(lat)#find closest lat, lon in era5
                int_lon=int(lon)
                lat_range=[x for x in lats_era5 if int_lat<=lat<=int_lat+1]
                lon_range=[x for x in lons_era5 if int_lon-1<=lon<=int_lon]#for negatives, int goes up
                dists_lat=[abs(x-lat) for x in lat_range]
                dists_lon=[abs(x-lon) for x in lon_range]
                zipped_lat=list(zip(np.arange(len(lat_range)),dists_lat))
                zipped_lon=list(zip(np.arange(len(lon_range)),dists_lon))
                lat_sorted=sorted(zipped_lat,key=lambda x:x[1])
                lon_sorted=sorted(zipped_lon,key=lambda x:x[1])
                closest_index_lat=lat_sorted[0][0]
                closest_index_lon=lon_sorted[0][0]
                lat_closest=lat_range[closest_index_lat]
                lon_closest=lon_range[closest_index_lon]

                extreme_ivt='no'
                ivt_square=ivt_mag_date.sel(latitude=np.arange(lat_closest-3*0.25,lat_closest+4*0.25,0.25),longitude=np.arange(lon_closest-4*0.25,lon_closest+5*0.25,0.25))
                ivt_extreme_square=ivt_square.where(ivt_square>=ivt_threshold)
                if ivt_extreme_square.sum().values>0:
                    extreme_ivt='yes'

                ar_square=ar_max_four.sel(latitude=np.arange(lat_closest-3*0.25,lat_closest+4*0.25,0.25),longitude=np.arange(lon_closest-4*0.25,lon_closest+5*0.25,0.25))
                ar_value=ar_square.max().values

                ar_present='no'
                if ar_value==1:
                    ar_present='yes'

                tc_present='no'
                extc_present='no'
                tc_linked_ivt='no'
                extc_linked_ivt='no'
                tc_linked_ar='no'
                extc_linked_ar='no'
                for date in date_range:
                    date=pd.to_datetime(date)

                    year=date.year
                    month=date.month
                    day=date.day
                    hour=date.hour

                    ar_select=ar_date_four.sel(time=date)
                    labeled_select=labeled_four.sel(time=date)
                    tc_trop_select=tc_trop.sel(time=date)
                    tc_nontrop_select=tc_nontrop.sel(time=date)

                    labeled_ar_check=labeled_select.where(ar_select==1)
                    labeled_ar_check=labeled_ar_check.where(labeled_ar_check>0)
                    ar_check_nums=labeled_ar_check.values.flatten().tolist()
                    ar_check_nums=[x for x in ar_check_nums if x>0]
                    ar_check_nums=list(set(ar_check_nums))

                    station_ivt_nums=labeled_four.sel(time=date,latitude=np.arange(lat_closest-3*0.25,lat_closest+4*0.25,0.25),longitude=np.arange(lon_closest-4*0.25,lon_closest+5*0.25,0.25))
                    station_ivt_nums=station_ivt_nums.values.flatten().tolist()
                    station_ivt_nums=[x for x in station_ivt_nums if x>0]
                    station_ivt_nums=list(set(station_ivt_nums))

                    labeled_tc_trop_check=labeled_select.where(tc_trop_select==1)
                    labeled_tc_trop_check=labeled_tc_trop_check.where(labeled_tc_trop_check>0)
                    tc_trop_check_nums=labeled_tc_trop_check.values.flatten().tolist()
                    tc_trop_check_nums=[x for x in tc_trop_check_nums if x>0]
                    tc_trop_check_nums=list(set(tc_trop_check_nums))

                    labeled_tc_nontrop_check=labeled_select.where(tc_nontrop_select==1)
                    labeled_tc_nontrop_check=labeled_tc_nontrop_check.where(labeled_tc_nontrop_check>0)
                    tc_nontrop_check_nums=labeled_tc_nontrop_check.values.flatten().tolist()
                    tc_nontrop_check_nums=[x for x in tc_nontrop_check_nums if x>0]
                    tc_nontrop_check_nums=list(set(tc_nontrop_check_nums))

                    tc_intersect=len([x for x in station_ivt_nums if x in tc_trop_check_nums])
                    extc_intersect=len([x for x in station_ivt_nums if x in tc_nontrop_check_nums])
                    ar_intersect=len([x for x in station_ivt_nums if x in ar_check_nums])

                    tc_ar_intersect=len([x for x in ar_check_nums if x in tc_trop_check_nums])
                    extc_ar_intersect=len([x for x in ar_check_nums if x in tc_nontrop_check_nums])

                    #if d==21:
                    #    print((station_ivt_nums,tc_trop_check_nums,ar_check_nums))

                    for j in range(len(station_ivt_nums)):
                        station_ivt_num=station_ivt_nums[j]
                        if station_ivt_num in tc_trop_check_nums:
                            tc_linked_ivt='yes'
                        if station_ivt_num in tc_nontrop_check_nums:
                            extc_linked_ivt='yes'
                        if station_ivt_num in ar_check_nums and station_ivt_num in tc_trop_check_nums:
                            tc_linked_ar='yes'
                        if station_ivt_num in ar_check_nums and station_ivt_num in tc_nontrop_check_nums:
                            extc_linked_ar='yes'

                    check_lat_tc=lat_tc.where(years_tc==year,drop=True)
                    check_lat_tc=check_lat_tc.where(months_tc==month,drop=True)
                    check_lat_tc=check_lat_tc.where(days_tc==day,drop=True)
                    check_lat_tc=check_lat_tc.where(hours_tc==hour,drop=True)
                    #TS,SS,EX,ET,
                    check_lat_tc_trop=check_lat_tc.where(nature_tc==1,drop=True)
                    check_lat_tc_nontrop=check_lat_tc.where(nature_tc!=1,drop=True)

                    check_lat_tc_trop_list=[]
                    for j in range(check_lat_tc_trop.storm.size):
                        check_lat_tc_trop_values=check_lat_tc_trop[j,:].values.tolist()
                        check_lat_tc_trop_values=[x for x in check_lat_tc_trop_values if x>0]
                        if len(check_lat_tc_trop_values)>0:
                            check_lat_tc_trop_value=check_lat_tc_trop_values[0]
                            check_lat_tc_trop_list.append(check_lat_tc_trop_value)

                    check_lat_tc_nontrop_list=[]
                    for j in range(check_lat_tc_nontrop.storm.size):
                        check_lat_tc_nontrop_values=check_lat_tc_nontrop[j,:].values.tolist()
                        check_lat_tc_nontrop_values=[x for x in check_lat_tc_nontrop_values if x>0]
                        if len(check_lat_tc_nontrop_values)>0:
                            check_lat_tc_nontrop_value=check_lat_tc_nontrop_values[0]
                            check_lat_tc_nontrop_list.append(check_lat_tc_nontrop_value)

                    check_lon_tc=lon_tc.where(years_tc==year,drop=True)
                    check_lon_tc=check_lon_tc.where(months_tc==month,drop=True)
                    check_lon_tc=check_lon_tc.where(days_tc==day,drop=True)
                    check_lon_tc=check_lon_tc.where(hours_tc==hour,drop=True)
                    check_lon_tc_trop=check_lon_tc.where(nature_tc==1,drop=True)
                    check_lon_tc_nontrop=check_lon_tc.where(nature_tc!=1,drop=True)

                    check_lon_tc_trop_list=[]
                    for j in range(check_lon_tc_trop.storm.size):
                        check_lon_tc_trop_values=check_lon_tc_trop[j,:].values.tolist()
                        check_lon_tc_trop_values=[x for x in check_lon_tc_trop_values if x>-1000]
                        if len(check_lon_tc_trop_values)>0:
                            check_lon_tc_trop_value=check_lon_tc_trop_values[0]
                            check_lon_tc_trop_list.append(check_lon_tc_trop_value)

                    check_lon_tc_nontrop_list=[]
                    for j in range(check_lon_tc_nontrop.storm.size):
                        check_lon_tc_nontrop_values=check_lon_tc_nontrop[j,:].values.tolist()
                        check_lon_tc_nontrop_values=[x for x in check_lon_tc_nontrop_values if x>-1000]
                        if len(check_lon_tc_nontrop_values)>0:
                            check_lon_tc_nontrop_value=check_lon_tc_nontrop_values[0]
                            check_lon_tc_nontrop_list.append(check_lon_tc_nontrop_value)

                    if len(check_lat_tc_trop_list)>0 and len(check_lon_tc_trop_list)>0:
                        for j in range(len(check_lat_tc_trop_list)):
                            lat_tc_single=check_lat_tc_trop_list[j]
                            lon_tc_single=check_lon_tc_trop_list[j]
                            distance=geopy.distance.distance((lat_tc_single,lon_tc_single),(lat,lon)).km
                            if distance<=500:
                                tc_present='yes'
                    if len(check_lat_tc_nontrop_list)>0 and len(check_lon_tc_nontrop_list)>0:
                        for j in range(len(check_lat_tc_nontrop_list)):
                            lat_tc_single=check_lat_tc_nontrop_list[j]
                            lon_tc_single=check_lon_tc_nontrop_list[j]
                            distance=geopy.distance.distance((lat_tc_single,lon_tc_single),(lat,lon)).km
                            if distance<=500:
                                extc_present='yes'

                if tc_present=='yes':
                    if ar_value==1:
                        #tc-ar combination
                        stations_tc_ar_combo.append(station)
                        lats_tc_ar_combo.append(lat)
                        lons_tc_ar_combo.append(lon)
                        obs_tc_ar_combo.append(obs_station)
                        dates_tc_ar_combo.append(date_save)
                    else:
                        #pure TC
                        stations_pure_tc.append(station)
                        lats_pure_tc.append(lat)
                        lons_pure_tc.append(lon)
                        obs_pure_tc.append(obs_station)
                        dates_pure_tc.append(date_save)
                elif ar_present=='yes':
                    if extc_present=='yes':
                        #TC remnant - AR combination
                        stations_tc_remnant_ar_combo.append(station)
                        lats_tc_remnant_ar_combo.append(lat)
                        lons_tc_remnant_ar_combo.append(lon)
                        obs_tc_remnant_ar_combo.append(obs_station)
                        dates_tc_remnant_ar_combo.append(date_save)
                    elif tc_linked_ar=='yes':
                        #if d==21:
                        #    print('TC-linked AR')
                        #TC-linked AR
                        stations_tc_linked_ar.append(station)
                        lats_tc_linked_ar.append(lat)
                        lons_tc_linked_ar.append(lon)
                        obs_tc_linked_ar.append(obs_station)
                        dates_tc_linked_ar.append(date_save)
                    elif extc_linked_ar=='yes':
                        #TC remnant - linked AR
                        stations_tc_remnant_linked_ar.append(station)
                        lats_tc_remnant_linked_ar.append(lat)
                        lons_tc_remnant_linked_ar.append(lon)
                        obs_tc_remnant_linked_ar.append(obs_station)
                        dates_tc_remnant_linked_ar.append(date_save)
                    else:
                        #Pure AR
                        stations_pure_ar.append(station)
                        lats_pure_ar.append(lat)
                        lons_pure_ar.append(lon)
                        obs_pure_ar.append(obs_station)
                        dates_pure_ar.append(date_save)
                elif extc_present=='yes':
                    #TC remnants
                    stations_tc_remnants.append(station)
                    lats_tc_remnants.append(lat)
                    lons_tc_remnants.append(lon)
                    obs_tc_remnants.append(obs_station)
                    dates_tc_remnants.append(date_save)
                elif extreme_ivt=='yes':
                    if tc_linked_ivt=='yes':
                        #TC-linked IVT
                        stations_tc_linked_ivt.append(station)
                        lats_tc_linked_ivt.append(lat)
                        lons_tc_linked_ivt.append(lon)
                        obs_tc_linked_ivt.append(obs_station)
                        dates_tc_linked_ivt.append(date_save)
                    elif extc_linked_ivt=='yes':
                        #TC remnant-linked IVT
                        stations_tc_remnant_linked_ivt.append(station)
                        lats_tc_remnant_linked_ivt.append(lat)
                        lons_tc_remnant_linked_ivt.append(lon)
                        obs_tc_remnant_linked_ivt.append(obs_station)
                        dates_tc_remnant_linked_ivt.append(date_save)
                    else:
                        #Pure extreme IVT
                        stations_pure_extreme_ivt.append(station)
                        lats_pure_extreme_ivt.append(lat)
                        lons_pure_extreme_ivt.append(lon)
                        obs_pure_extreme_ivt.append(obs_station)
                        dates_pure_extreme_ivt.append(date_save)
                else:
                    #Unspecified
                    stations_other.append(station)
                    lats_other.append(lat)
                    lons_other.append(lon)
                    obs_other.append(obs_station)
                    dates_other.append(date_save)

    dk=xr.Dataset()
    dk['lats_pure_ar']=(('time_pure_ar'),lats_pure_ar)
    dk['lons_pure_ar']=(('time_pure_ar'),lons_pure_ar)
    dk['obs_pure_ar']=(('time_pure_ar'),obs_pure_ar)
    dk['dates_pure_ar']=(('time_pure_ar'),dates_pure_ar)
    dk['stations_pure_ar']=(('time_pure_ar'),stations_pure_ar)

    dk['lats_pure_tc']=(('time_pure_tc'),lats_pure_tc)
    dk['lons_pure_tc']=(('time_pure_tc'),lons_pure_tc)
    dk['obs_pure_tc']=(('time_pure_tc'),obs_pure_tc)
    dk['dates_pure_tc']=(('time_pure_tc'),dates_pure_tc)
    dk['stations_pure_tc']=(('time_pure_tc'),stations_pure_tc)

    dk['lats_pure_extreme_ivt']=(('time_pure_extreme_ivt'),lats_pure_extreme_ivt)
    dk['lons_pure_extreme_ivt']=(('time_pure_extreme_ivt'),lons_pure_extreme_ivt)
    dk['obs_pure_extreme_ivt']=(('time_pure_extreme_ivt'),obs_pure_extreme_ivt)
    dk['dates_pure_extreme_ivt']=(('time_pure_extreme_ivt'),dates_pure_extreme_ivt)
    dk['stations_pure_extreme_ivt']=(('time_pure_extreme_ivt'),stations_pure_extreme_ivt)

    dk['lats_tc_linked_ar']=(('time_tc_linked_ar'),lats_tc_linked_ar)
    dk['lons_tc_linked_ar']=(('time_tc_linked_ar'),lons_tc_linked_ar)
    dk['obs_tc_linked_ar']=(('time_tc_linked_ar'),obs_tc_linked_ar)
    dk['dates_tc_linked_ar']=(('time_tc_linked_ar'),dates_tc_linked_ar)
    dk['stations_tc_linked_ar']=(('time_tc_linked_ar'),stations_tc_linked_ar)

    dk['lats_tc_remnant_linked_ar']=(('time_tc_remnant_linked_ar'),lats_tc_remnant_linked_ar)
    dk['lons_tc_remnant_linked_ar']=(('time_tc_remnant_linked_ar'),lons_tc_remnant_linked_ar)
    dk['obs_tc_remnant_linked_ar']=(('time_tc_remnant_linked_ar'),obs_tc_remnant_linked_ar)
    dk['dates_tc_remnant_linked_ar']=(('time_tc_remnant_linked_ar'),dates_tc_remnant_linked_ar)
    dk['stations_tc_remnant_linked_ar']=(('time_tc_remnant_linked_ar'),stations_tc_remnant_linked_ar)

    dk['lats_tc_linked_ivt']=(('time_tc_linked_ivt'),lats_tc_linked_ivt)
    dk['lons_tc_linked_ivt']=(('time_tc_linked_ivt'),lons_tc_linked_ivt)
    dk['obs_tc_linked_ivt']=(('time_tc_linked_ivt'),obs_tc_linked_ivt)
    dk['dates_tc_linked_ivt']=(('time_tc_linked_ivt'),dates_tc_linked_ivt)
    dk['stations_tc_linked_ivt']=(('time_tc_linked_ivt'),stations_tc_linked_ivt)

    dk['lats_tc_remnant_linked_ivt']=(('time_tc_remnant_linked_ivt'),lats_tc_remnant_linked_ivt)
    dk['lons_tc_remnant_linked_ivt']=(('time_tc_remnant_linked_ivt'),lons_tc_remnant_linked_ivt)
    dk['obs_tc_remnant_linked_ivt']=(('time_tc_remnant_linked_ivt'),obs_tc_remnant_linked_ivt)
    dk['dates_tc_remnant_linked_ivt']=(('time_tc_remnant_linked_ivt'),dates_tc_remnant_linked_ivt)
    dk['stations_tc_remnant_linked_ivt']=(('time_tc_remnant_linked_ivt'),stations_tc_remnant_linked_ivt)

    dk['lats_tc_remnants']=(('time_tc_remnants'),lats_tc_remnants)
    dk['lons_tc_remnants']=(('time_tc_remnants'),lons_tc_remnants)
    dk['obs_tc_remnants']=(('time_tc_remnants'),obs_tc_remnants)
    dk['dates_tc_remnants']=(('time_tc_remnants'),dates_tc_remnants)
    dk['stations_tc_remnants']=(('time_tc_remnants'),stations_tc_remnants)

    dk['lats_tc_ar_combo']=(('time_tc_ar_combo'),lats_tc_ar_combo)
    dk['lons_tc_ar_combo']=(('time_tc_ar_combo'),lons_tc_ar_combo)
    dk['obs_tc_ar_combo']=(('time_tc_ar_combo'),obs_tc_ar_combo)
    dk['dates_tc_ar_combo']=(('time_tc_ar_combo'),dates_tc_ar_combo)
    dk['stations_tc_ar_combo']=(('time_tc_ar_combo'),stations_tc_ar_combo)

    dk['lats_tc_remnant_ar_combo']=(('time_tc_remnant_ar_combo'),lats_tc_remnant_ar_combo)
    dk['lons_tc_remnant_ar_combo']=(('time_tc_remnant_ar_combo'),lons_tc_remnant_ar_combo)
    dk['obs_tc_remnant_ar_combo']=(('time_tc_remnant_ar_combo'),obs_tc_remnant_ar_combo)
    dk['dates_tc_remnant_ar_combo']=(('time_tc_remnant_ar_combo'),dates_tc_remnant_ar_combo)
    dk['stations_tc_remnant_ar_combo']=(('time_tc_remnant_ar_combo'),stations_tc_remnant_ar_combo)

    dk['lats_other']=(('time_other'),lats_other)
    dk['lons_other']=(('time_other'),lons_other)
    dk['obs_other']=(('time_other'),obs_other)
    dk['dates_other']=(('time_other'),dates_other)
    dk['stations_other']=(('time_other'),stations_other)


    dk.coords['time_pure_ar']=np.arange(len(lats_pure_ar))
    dk.coords['time_pure_tc']=np.arange(len(lats_pure_tc))
    dk.coords['time_pure_extreme_ivt']=np.arange(len(lats_pure_extreme_ivt))
    dk.coords['time_tc_linked_ar']=np.arange(len(lats_tc_linked_ar))
    dk.coords['time_tc_remnant_linked_ar']=np.arange(len(lats_tc_remnant_linked_ar))
    dk.coords['time_tc_linked_ivt']=np.arange(len(lats_tc_linked_ivt))
    dk.coords['time_tc_remnant_linked_ivt']=np.arange(len(lats_tc_remnant_linked_ivt))
    dk.coords['time_tc_remnants']=np.arange(len(lats_tc_remnants))
    dk.coords['time_tc_ar_combo']=np.arange(len(lats_tc_ar_combo))
    dk.coords['time_tc_remnant_ar_combo']=np.arange(len(lats_tc_remnant_ar_combo))
    dk.coords['time_other']=np.arange(len(lats_other))

    try:
        os.remove(dir+'climo_neusa_ar_'+season+'warm_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir+'climo_neusa_ar_'+season+'warm_'+str(year)+'.nc',mode='w',format='NETCDF4')
    dk.close()